Read Data

In [142]:
import numpy as np
import pandas as pd

df = pd.read_csv('./weatherAUS.csv')
print('Size of weather data frame is :',df.shape)
#Let us see how our data looks like!
df[0:5]

Size of weather data frame is : (145460, 24)


Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0  2008-12-01   Albury     13.4     22.9       0.6          NaN       NaN   
1  2008-12-02   Albury      7.4     25.1       0.0          NaN       NaN   
2  2008-12-03   Albury     12.9     25.7       0.0          NaN       NaN   
3  2008-12-04   Albury      9.2     28.0       0.0          NaN       NaN   
4  2008-12-05   Albury     17.5     32.3       1.0          NaN       NaN   

  WindGustDir  WindGustSpeed WindDir9am  ... Humidity3pm  Pressure9am  \
0           W           44.0          W  ...        22.0       1007.7   
1         WNW           44.0        NNW  ...        25.0       1010.6   
2         WSW           46.0          W  ...        30.0       1007.6   
3          NE           24.0         SE  ...        16.0       1017.6   
4           W           41.0        ENE  ...        33.0       1010.8   

   Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm  RainToday  RISK_MM  \
0       1007.1       8.0       NaN     16.9     21.8         No      0.0   
1       1007.8       NaN       NaN     17.2     24.3         No      0.0   
2       1008.7       NaN       2.0     21.0     23.2         No      0.0   
3       1012.8       NaN       NaN     18.1     26.5         No      1.0   
4       1006.0       7.0       8.0     17.8     29.7         No      0.2   

   RainTomorrow  
0            No  
1            No  
2            No  
3            No  
4            No  

[5 rows x 24 columns]

In [143]:
df.shape

(145460, 24)

Drop RISK_MM column

In [144]:
df.drop(columns=['RISK_MM'], axis = 1, inplace = True)

In [145]:
df.shape

(145460, 23)

# Data Pruning
### 1. Deal with Nan & null values
Count null values

In [146]:
df.isna().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

Few columns such as Sunshine, Evaporation, Cloud9am, Cloud3pm have greater than 50000 null values which is almost 33% or more. We can either **ignore** these columns or **fill** the NaN values with some dummy values. Here, we will these drop the features.  
 
TODO: DO PCA or any other method

In [147]:
## Drop Null valued columns
df.drop(columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am'], axis=1, inplace = True)
print(df.columns.values)

['Date' 'Location' 'MinTemp' 'MaxTemp' 'Rainfall' 'WindGustDir'
 'WindGustSpeed' 'WindDir9am' 'WindDir3pm' 'WindSpeed9am' 'WindSpeed3pm'
 'Humidity9am' 'Humidity3pm' 'Pressure9am' 'Pressure3pm' 'Temp9am'
 'Temp3pm' 'RainToday' 'RainTomorrow']


Other columns do have null values in them, we could either ignore the rows or substitute them mean values of the column. We can try both approaches, first dropping the rows.

1. Some columns as RainToday, Rain Tomorrow are non-numeric columns, while substituting the NaN values by mean, we can first change the values to 0 and 1. 

*Dropping rows with Rain Tomorrow = NA values as these rows not useful intraining or testing.*

In [148]:
df.dropna(axis=0, subset=['RainTomorrow'], inplace=True)

### 1.1 Handling binary data

In [149]:
df['RainToday'].replace({'No': 0, 'Yes': 1},inplace = True)
df['RainTomorrow'].replace({'No': 0, 'Yes': 1},inplace = True)

Now susbtituing values

In [150]:
# Prefill missing values with mean
df.fillna(df.mean(), inplace=True)
# Drop rows with null values
# df.dropna(how='any', inplace=True)
# df.shape

#### Should we do Z-score to remove outliers?

Currently, we will keep the outliers, we can also check with removing the outliers.

### 2. Handle Date value
We must change date variable and split it to month, year, date. It would be interesting to check if month, year & day possibly have any correlation with the result.

In [151]:
df['Date'] = pd.to_datetime(df['Date'])

# Extract date components
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# Drop date column
df.drop('Date', axis=1, inplace = True)

### 3. Handling categorical data

There are number of columns which have categorical data for eg:- WindGustDir, Location. This data needs to be represented such as it is palapable to the models. Hence, we encode these data columns as one hot vector, by adding an additional columns.

In [152]:
categorical_columns = ['WindGustDir', 'WindDir3pm', 'WindDir9am', 'Location']
df = pd.get_dummies(df, columns=categorical_columns) #, dummy_na=True
df.info()
# TODO: Check categorical data for unique values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 145458
Columns: 113 entries, MinTemp to Location_Woomera
dtypes: float64(13), int64(3), uint8(97)
memory usage: 31.6 MB


# Feature Scaling
Feature domination for eg:- Pressure might dominate temperature from above sample.
There are few scaling methods that can scale data reduce feature domination. 

1. Standard Scalar: scales features to -1 to 1 range with 0 mean.
2. MinMax Scalar: scales to 0 to 1 range.
3. QuantileTransformer: Fits data onto a normal distribution and reduces the effects of outliers
4. PowerTransformer: Fits data onto a Gaussian distribution.

Before, we proceed, it would be a good excercise to get some statistical analysis on features.

In [153]:
df.describe()

MinTemp        MaxTemp       Rainfall  WindGustSpeed  \
count  142193.000000  142193.000000  142193.000000  142193.000000   
mean       12.186400      23.226784       2.349974      39.984292   
std         6.388924       7.109554       8.423217      13.138385   
min        -8.500000      -4.800000       0.000000       6.000000   
25%         7.600000      17.900000       0.000000      31.000000   
50%        12.000000      22.700000       0.000000      39.000000   
75%        16.800000      28.200000       0.800000      46.000000   
max        33.900000      48.100000     371.000000     135.000000   

        WindSpeed9am   WindSpeed3pm    Humidity9am    Humidity3pm  \
count  142193.000000  142193.000000  142193.000000  142193.000000   
mean       14.001988      18.637576      68.843810      51.482606   
std         8.851082       8.721551      18.932077      20.532065   
min         0.000000       0.000000       0.000000       0.000000   
25%         7.000000      13.000000      57.000000      37.000000   
50%        13.000000      18.637576      70.000000      51.482606   
75%        19.000000      24.000000      83.000000      65.000000   
max       130.000000      87.000000     100.000000     100.000000   

         Pressure9am    Pressure3pm  ...  Location_Townsville  \
count  142193.000000  142193.000000  ...        142193.000000   
mean     1017.653758    1015.258204  ...             0.021330   
std         6.746248       6.681788  ...             0.144483   
min       980.500000     977.100000  ...             0.000000   
25%      1013.500000    1011.000000  ...             0.000000   
50%      1017.653758    1015.258204  ...             0.000000   
75%      1021.800000    1019.400000  ...             0.000000   
max      1041.000000    1039.600000  ...             1.000000   

       Location_Tuggeranong  Location_Uluru  Location_WaggaWagga  \
count         142193.000000   142193.000000        142193.000000   
mean               0.021084        0.010697             0.020929   
std                0.143665        0.102871             0.143148   
min                0.000000        0.000000             0.000000   
25%                0.000000        0.000000             0.000000   
50%                0.000000        0.000000             0.000000   
75%                0.000000        0.000000             0.000000   
max                1.000000        1.000000             1.000000   

       Location_Walpole  Location_Watsonia  Location_Williamtown  \
count     142193.000000      142193.000000         142193.000000   
mean           0.019825           0.021091              0.017954   
std            0.139400           0.143688              0.132786   
min            0.000000           0.000000              0.000000   
25%            0.000000           0.000000              0.000000   
50%            0.000000           0.000000              0.000000   
75%            0.000000           0.000000              0.000000   
max            1.000000           1.000000              1.000000   

       Location_Witchcliffe  Location_Wollongong  Location_Woomera  
count         142193.000000        142193.000000     142193.000000  
mean               0.020761             0.020979          0.021028  
std                0.142582             0.143313          0.143477  
min                0.000000             0.000000          0.000000  
25%                0.000000             0.000000          0.000000  
50%                0.000000             0.000000          0.000000  
75%                0.000000             0.000000          0.000000  
max                1.000000             1.000000          1.000000  

[8 rows x 113 columns]

#### Inference :
It is clear from the std deviation that most of the data points are well surrounded around the mean. Scaling techniques like MinMacScalar might accidently include the influence of outliers. Nonetheless, we experiment with  
1. Standard Scalar
2. Quantile Transformer
3. MinMax Scalar

Currently, as data contains one hot encoding it would be great to use MinMaxScalar. But we could essentially apply different transformation on categorical versus numerical data.

For simplicity, we first experiment with MinMaxScalar.

In [154]:
from sklearn import preprocessing

# scaler = preprocessing.StandardScaler()
# scaler = preprocessing.QuantileTransformer() #(ignore_implicit_zeros=True) if used on one-hot encoded
scaler = preprocessing.MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


### Scaled Data

In [155]:
df.iloc[0:10]

MinTemp   MaxTemp  Rainfall  WindGustSpeed  WindSpeed9am  WindSpeed3pm  \
0  0.516509  0.523629  0.001617       0.294574      0.153846      0.275862   
1  0.375000  0.565217  0.000000       0.294574      0.030769      0.252874   
2  0.504717  0.576560  0.000000       0.310078      0.146154      0.298851   
3  0.417453  0.620038  0.000000       0.139535      0.084615      0.103448   
4  0.613208  0.701323  0.002695       0.271318      0.053846      0.229885   
5  0.544811  0.652174  0.000539       0.387597      0.146154      0.275862   
6  0.537736  0.563327  0.000000       0.341085      0.153846      0.275862   
7  0.382075  0.595463  0.000000       0.224806      0.046154      0.195402   
8  0.429245  0.693762  0.000000       0.573643      0.053846      0.321839   
9  0.509434  0.659735  0.003774       0.170543      0.115385      0.126437   

   Humidity9am  Humidity3pm  Pressure9am  Pressure3pm  ...  \
0         0.71         0.22     0.449587       0.4800  ...   
1         0.44         0.25     0.497521       0.4912  ...   
2         0.38         0.30     0.447934       0.5056  ...   
3         0.45         0.16     0.613223       0.5712  ...   
4         0.82         0.33     0.500826       0.4624  ...   
5         0.55         0.23     0.474380       0.4528  ...   
6         0.49         0.19     0.480992       0.4976  ...   
7         0.48         0.19     0.543802       0.5280  ...   
8         0.42         0.09     0.469421       0.4240  ...   
9         0.58         0.27     0.438017       0.4576  ...   

   Location_Townsville  Location_Tuggeranong  Location_Uluru  \
0                  0.0                   0.0             0.0   
1                  0.0                   0.0             0.0   
2                  0.0                   0.0             0.0   
3                  0.0                   0.0             0.0   
4                  0.0                   0.0             0.0   
5                  0.0                   0.0             0.0   
6                  0.0                   0.0             0.0   
7                  0.0                   0.0             0.0   
8                  0.0                   0.0             0.0   
9                  0.0                   0.0             0.0   

   Location_WaggaWagga  Location_Walpole  Location_Watsonia  \
0                  0.0               0.0                0.0   
1                  0.0               0.0                0.0   
2                  0.0               0.0                0.0   
3                  0.0               0.0                0.0   
4                  0.0               0.0                0.0   
5                  0.0               0.0                0.0   
6                  0.0               0.0                0.0   
7                  0.0               0.0                0.0   
8                  0.0               0.0                0.0   
9                  0.0               0.0                0.0   

   Location_Williamtown  Location_Witchcliffe  Location_Wollongong  \
0                   0.0                   0.0                  0.0   
1                   0.0                   0.0                  0.0   
2                   0.0                   0.0                  0.0   
3                   0.0                   0.0                  0.0   
4                   0.0                   0.0                  0.0   
5                   0.0                   0.0                  0.0   
6                   0.0                   0.0                  0.0   
7                   0.0                   0.0                  0.0   
8                   0.0                   0.0                  0.0   
9                   0.0                   0.0                  0.0   

   Location_Woomera  
0               0.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  
5               0.0  
6               0.0  
7               0.0  
8               0.0  
9               0.0  

[10 rows x 113 columns]

# Feature Selection
There are multitude of methods for feature selection.
1. Univariate feature selection
2. Pearson's co-efficient
3. PCA
4. Recurssive feature elimination with cross validation

In [156]:
from sklearn.feature_selection import SelectKBest, chi2
X = df.loc[:,df.columns!='RainTomorrow']
y = df[['RainTomorrow']]
selector = SelectKBest(chi2, k=40)
selector.fit(X, y)
X_new = selector.transform(X)
print(X.columns[selector.get_support(indices=True)]) #top 3 columns

Index(['MaxTemp', 'Rainfall', 'WindGustSpeed', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Temp3pm', 'RainToday', 'WindGustDir_E',
       'WindGustDir_ENE', 'WindGustDir_ESE', 'WindGustDir_N',
       'WindGustDir_NNW', 'WindGustDir_NW', 'WindGustDir_WNW', 'WindDir3pm_E',
       'WindDir3pm_ESE', 'WindDir3pm_N', 'WindDir3pm_NNW', 'WindDir3pm_NW',
       'WindDir3pm_WNW', 'WindDir9am_E', 'WindDir9am_ESE', 'WindDir9am_N',
       'WindDir9am_NNW', 'WindDir9am_NW', 'WindDir9am_SE',
       'Location_AliceSprings', 'Location_Cairns', 'Location_Cobar',
       'Location_Dartmoor', 'Location_Mildura', 'Location_Moree',
       'Location_MountGambier', 'Location_NorfolkIsland', 'Location_Portland',
       'Location_Uluru', 'Location_Walpole', 'Location_Woomera'],
      dtype='object')


# Split into Train / Test data

In [158]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

## Training
1. Select models and apply hyper parameters.
2. Use ross validation (k-fold).
//explain
3. Use GridSearch CV
//explain

## Logistic Regression
### Train
About Hyper-Parameters
1. Trying both l1 & l2 regularization penalty
2. Balanced weights as classes are unbalanced

In [177]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
#clf_logreg = LogisticRegression(random_state=0, class_weight='balanced', C=100)
#clf_logreg.fit(X_train,y_train)

## Hyper parameters
"""
hyperparameters = dict(
    C=np.logspace( 2, 4, 6),     # regularization penalty 
    penalty=['l1', 'l2'],        # regularization hyperparameter
    #tol=np.logspace( 2, 4),
    #solver=['liblinear', 'saga'],# solvers to use
    class_weight=['balanced'],
    n_jobs=-1)                   # use all cpus 
"""
hyperparameters=dict()           # dry run

clf = GridSearchCV(LogisticRegression(), hyperparameters, cv=3, verbose=0)

#### Train and print best model

In [178]:
best_model = clf.fit(X_train, y_train)
print("Best parameters set found on development set:")
print()
print(best_model.best_params_)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

Best parameters set found on development set:

{}


### Test & F1 score, Accuracy

In [179]:
from sklearn.metrics import classification_report
y_pred = best_model.predict(X_test)
score = classification_report(y_test,y_pred, labels=[0.0, 1.0])
print('Accuracy :',score)

Accuracy :               precision    recall  f1-score   support

         0.0       0.87      0.95      0.90     27496
         1.0       0.73      0.50      0.59      8053

   micro avg       0.85      0.85      0.85     35549
   macro avg       0.80      0.72      0.75     35549
weighted avg       0.84      0.85      0.83     35549



## Decision Trees
### Train
About Hyper-Parameters


In [187]:
#Decision Tree Classifier
from sklearn.ensemble import GradientBoostingClassifier
#from imblearn.pipeline import Pipeline
#from imblearn.over_sampling import SMOTE

## Hyper parameters
hyperparameters = dict()

#model = Pipeline([
#        ('sampling', SMOTE()),  ## Dry run
#        ('classification', GradientBoostingClassifier())
#    ])
model = GradientBoostingClassifier()
clf = GridSearchCV(model, hyperparameters, cv=3, verbose=0)

### Train and print best model

In [188]:
best_model = clf.fit(X_train, y_train)
print("Best parameters set found on development set:")
print()
print(best_model.best_params_)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validati

Best parameters set found on development set:

{}


### Test & F1 score, Accuracy

In [189]:
from sklearn.metrics import classification_report
y_pred = best_model.predict(X_test)
score = classification_report(y_test,y_pred, labels=[0.0, 1.0])
print('Accuracy :',score)

Accuracy :               precision    recall  f1-score   support

         0.0       0.86      0.95      0.91     27496
         1.0       0.75      0.48      0.59      8053

   micro avg       0.85      0.85      0.85     35549
   macro avg       0.80      0.72      0.75     35549
weighted avg       0.84      0.85      0.83     35549



**Thus, GradientBoostedTrees have a higher recall for class 0 ( not raining ) and lower f1score for label 1.0 than LogisticReegrssion.**